In [141]:
import pandas as pd
pd.set_option('max_colwidth', 35)
import re
import certifi
import urllib3
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [122]:
df = pd.read_csv("Openrice_Transformed_V2.csv")

### Summary: Identify the most important sentence

In [165]:
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict


def summarize_reviews(s):
    
    sent_to = sent_tokenize(str(s.lower()))
    word_to = word_tokenize(str(s.lower()))
    
    _stopwords = set(stopwords.words("english") + list(punctuation))
    word_clean = [w for w in word_to if w not in _stopwords]
    freq = FreqDist(word_clean)
    # nlargest(10, freq, key = freq.get)

    # identify important sentences
    ranking = defaultdict(int)

    for i, sen in enumerate(sent_to):
        for w in word_tokenize(sen.lower()):
            if w in freq:
                ranking[i] += freq[w]

    sents_index = nlargest(1, ranking, key = ranking.get)
    return [sent_to[j] for j in sorted(sents_index)]

In [166]:
# https://stackoverflow.com/questions/38119954/tokenizing-words-into-a-new-column-in-a-pandas-dataframe
addwords = lambda x: summarize_reviews(x)

In [167]:
df["summary"] = df["Reviews"].apply(addwords)

In [168]:
df["summary"][7]

['i fount it interesting they served it on a thin piece of lettuce like how it was served in the old days before steaming it in paper like it is now.0stir fried shanghai noodles.']

In [164]:
df.head()

,Restaurant_ID,Restaurant_Name,District,Cuisine,Rating_Average,Cost_Average,Wait_Average,Reviews,Restaurant_Link,summary
0,0,CHIRASHI,Wan Chai,Japanese,3.7,488.0,0.0,0Chirashi is a popular Japanese...,www.openrice.com/en/hongkong/re...,[e don has two layers with the ...
1,1,X,Causeway Bay,Chinese,3.6,60.0,0.0,There are a number of Ki's Roas...,www.openrice.com/en/hongkong/re...,[ere are a number of ki's roast...
2,2,10 Shanghai,Causeway Bay,Chinese,3.2,120.0,NaN,1957 & Company (company behind ...,www.openrice.com/en/hongkong/re...,[t this turns out to be more li...
3,3,1935 Restaurant,Central,Chinese,4.8,270.0,NaN,The food is delicious!!! We wer...,www.openrice.com/en/hongkong/re...,[ i have to choose my favorites...
4,4,1963,Wan Chai,Chinese,3.6,113.3,2.0,00This shop in Wanchai is locat...,www.openrice.com/en/hongkong/re...,[ '0entering this place was lik...


In [137]:
def get_recommendations_sum(title, cosine_sim = cosine_sim):
    
    # get the index of the movie that matches the title
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores,
                       key = lambda x: x[1],
                       reverse = True)
    
    sim_scores = sim_scores[1:11]
    
    rest_indices = [i[0] for i in sim_scores]
    
    return df[["Restaurant_Name","Restaurant_Link","District","Rating_Average","Wait_Average","summary"]].iloc[rest_indices]

In [142]:
get_recommendations_sum("Five Guys", cosine_sim2)

,Restaurant_Name,Restaurant_Link,District,Rating_Average,Wait_Average,summary
73,Burger Joys,www.openrice.com/en/hongkong/re...,Wan Chai,4.1,2.0,[the burger patty had just the ...
611,Shake'em Buns,www.openrice.com/en/hongkong/re...,Central,3.6,7.0,[!hoola gal burger (combo with ...
295,Honbo,www.openrice.com/en/hongkong/re...,Wan Chai,3.4,3.0,[while im personally not a fan ...
95,Caliburger,www.openrice.com/en/hongkong/re...,Wan Chai,2.6,3.0,"[at the same time, i couldn't h..."
734,The Butchers Club,www.openrice.com/en/hongkong/re...,Wan Chai,3.1,10.0,[beyond meat vegetarian burger$...
75,Burgeroom,www.openrice.com/en/hongkong/re...,Causeway Bay,2.8,7.0,"["", 'i have not seen so many ch..."
71,Burger Circus,www.openrice.com/en/hongkong/re...,Central,3.6,4.0,[definitely recommend if you wa...
72,Burger Home,www.openrice.com/en/hongkong/re...,Wan Chai,3.8,13.0,"[here in burger home, this was ..."
250,Gourmet Burger Union,www.openrice.com/en/hongkong/re...,Central,2.8,2.0,"[to be fair, the taste was good..."
58,Beef & Liberty,www.openrice.com/en/hongkong/re...,Wan Chai,3.9,9.0,"[the grain-fed ones), we were s..."


In [143]:
df.to_csv("Openrice_Summary_V1.csv")